In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_new, kernels_reparametrization_space as kernels_repar_space
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

Load monthly data

In [2]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [15]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

for day_index in range(31):
  
    hour_start_index = day_index * 8
    #hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=None,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
print(daily_aggregated_tensors[1][:10])

torch.Size([18126, 4])
tensor([[  4.9760, 132.9840, 267.0307,  45.0000],
        [  4.9760, 132.9210, 264.0250,  45.0000],
        [  4.9760, 132.8580, 258.8635,  45.0000],
        [  4.9760, 132.7950, 263.3982,  45.0000],
        [  4.9760, 132.7320, 272.0078,  45.0000],
        [  4.9760, 132.6690, 268.6431,  45.0000],
        [  4.9760, 132.6060, 267.8075,  45.0000],
        [  4.9760, 132.5430, 267.0912,  45.0000],
        [  4.9760, 132.4800, 266.1405,  45.0000],
        [  4.9760, 132.4170, 260.4594,  45.0000]])


# swap columns

In [94]:
daily_aggregated_tensors2 = [] 
daily_hourly_maps2 = []      
for day_index in range(31):
    hour_start_index = day_index * 8
    hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data_keep_ori(
    df_map, 
    hour_indices, 
    ord_mm= None,  # or just omit it
    dtype=torch.float # or just omit it 
    )
    day_aggregated_tensor[:,0], day_aggregated_tensor[:,1] = day_aggregated_tensor[:,4] , day_aggregated_tensor[:,5]


    for map in day_hourly_map.values():
        # This single line performs the swap safely and efficiently
        map[:,0], map[:,1], map[:,4], map[:,5] = map[:,4], map[:,5], map[:,0], map[:,1]
        
    daily_aggregated_tensors2.append( day_aggregated_tensor )
    daily_hourly_maps2.append( day_hourly_map )

print(daily_aggregated_tensors2[1][:10])


tensor([[  4.9764, 132.9858, 267.0307,  45.0000,   4.9764, 132.9858],
        [  4.9765, 132.9230, 264.0250,  45.0000,   4.9765, 132.9230],
        [  4.9766, 132.8600, 258.8635,  45.0000,   4.9766, 132.8600],
        [  4.9768, 132.7975, 263.3982,  45.0000,   4.9768, 132.7975],
        [  4.9769, 132.7346, 272.0078,  45.0000,   4.9769, 132.7346],
        [  4.9769, 132.6719, 268.6431,  45.0000,   4.9769, 132.6719],
        [  4.9772, 132.6088, 267.8075,  45.0000,   4.9772, 132.6088],
        [  4.9774, 132.5458, 267.0912,  45.0000,   4.9774, 132.5458],
        [  4.9776, 132.4833, 266.1405,  45.0000,   4.9776, 132.4833],
        [  4.9776, 132.4204, 260.4594,  45.0000,   4.9776, 132.4204]])


In [95]:
da_idx = 8

lat = [daily_aggregated_tensors[da_idx][:,0], daily_aggregated_tensors2[da_idx][:,0]]
lon = [daily_aggregated_tensors[da_idx][:,1], daily_aggregated_tensors2[da_idx][:,1]]

In [96]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# --- Assuming this is your data based on the example ---
tensor_original = lat[0]

tensor_grid = lat[1]


# --- Analysis ---
original_np = tensor_original.detach().cpu().numpy()
grid_np = tensor_grid.detach().cpu().numpy()

# 1. Calculate the deviation
deviation = original_np - grid_np

# 2. Define your threshold
threshold = 0.022

# 3. Find all absolute deviations greater than the threshold
# np.abs() is crucial to catch both large positive and large negative diffs
large_deviations_mask = np.abs(deviation) > threshold

# 4. Count them
count = np.sum(large_deviations_mask)

# --- Report Results ---
total_points = len(deviation)
percentage = (count / total_points) * 100

print(f"--- Fine-Scale Deviation Report ---")
print(f"Threshold: > {threshold}")
print(f"Total data points: {total_points}")
print(f"Points with large deviation: {count}")
print(f"Percentage with large deviation: {percentage:.2f}%")

# Optional: Show the actual large deviation values
if count > 0:
    print("\nValues of the large deviations:")
    print(deviation[large_deviations_mask])
else:
    print("\nNo deviations found above the threshold.")

--- Fine-Scale Deviation Report ---
Threshold: > 0.022
Total data points: 18126
Points with large deviation: 147
Percentage with large deviation: 0.81%

Values of the large deviations:
[-0.04604769 -0.04634285 -0.02498245 -0.02498245 -0.0251441  -0.04697132
 -0.0689826  -0.0689826   0.06156063 -0.04745197  0.08293152 -0.02643919
 -0.02643919 -0.04833174 -0.04833174  0.03893137  0.03893137 -0.09233189
  0.03818488  0.05994868  0.08144546 -0.04960227 -0.02844429  0.03744555
  0.03744555 -0.02810502 -0.05088878 -0.05098557  0.058815    0.03657699
  0.05772376  0.03563738  0.03563738 -0.0297122   0.03652239  0.0363369
  0.0363369   0.03527045  0.0358212  -0.02970552 -0.05319834 -0.02962255
  0.03337431 -0.03007603 -0.05232406 -0.03097391 -0.03164864  0.03480363
  0.03441644  0.03441644 -0.0529213  -0.03138566 -0.03138566 -0.03175521
 -0.07564878 -0.03213763 -0.09692121 -0.07538557  0.05510449 -0.07613754
  0.09855103 -0.03304458 -0.03304458 -0.05473518 -0.05473518 -0.05473518
 -0.03324294 

In [86]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# --- Assuming this is your data based on the example ---
tensor_original = lon[0]

tensor_grid = lon[1]


# --- Analysis ---
original_np = tensor_original.detach().cpu().numpy()
grid_np = tensor_grid.detach().cpu().numpy()

# 1. Calculate the deviation
deviation = original_np - grid_np

# 2. Define your threshold
threshold = 0.031

# 3. Find all absolute deviations greater than the threshold
# np.abs() is crucial to catch both large positive and large negative diffs
large_deviations_mask = np.abs(deviation) > threshold

# 4. Count them
count = np.sum(large_deviations_mask)

# --- Report Results ---
total_points = len(deviation)
percentage = (count / total_points) * 100

print(f"--- Fine-Scale Deviation Report ---")
print(f"Threshold: > {threshold}")
print(f"Total data points: {total_points}")
print(f"Points with large deviation: {count}")
print(f"Percentage with large deviation: {percentage:.2f}%")

# Optional: Show the actual large deviation values
if count > 0:
    print("\nValues of the large deviations:")
    print(deviation[large_deviations_mask])
else:
    print("\nNo deviations found above the threshold.")

--- Fine-Scale Deviation Report ---
Threshold: > 0.031
Total data points: 18126
Points with large deviation: 130
Percentage with large deviation: 0.72%

Values of the large deviations:
[-0.08123779  0.04386902  0.04299927 -0.03128052 -0.08010864  0.04386902
  0.04354858 -0.08010864  0.04386902 -0.03125    -0.07954407  0.04441833
  0.03120422  0.04727173  0.03114319  0.04759216  0.04727173  0.04782104
 -0.07011414  0.05357361 -0.0697937   0.05451965  0.03118896 -0.06820679
  0.05514526  0.03126526  0.05545044 -0.03102112  0.05670166  0.05371094
  0.05700684  0.05371094  0.03111267 -0.06781006 -0.07009888  0.05548096
 -0.03100586 -0.06718445 -0.0697937   0.05548096 -0.06761169  0.05607605
 -0.06700134  0.03120422 -0.03100586  0.06021118 -0.03103638  0.03128052
  0.06112671 -0.03137207  0.03133392 -0.03125     0.03108978 -0.03117371
  0.03124237 -0.03121185  0.03101349 -0.03103638 -0.03101349  0.03126526
 -0.03134155  0.03125    -0.03137207 -0.03127289  0.03152466  0.03112793
 -0.03102112

Hyper parameters

In [ ]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=100

In [ ]:
# --- End Placeholders ---

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                
    init_phi1 = init_sigmasq * init_phi2            
    init_phi3 = (init_range_lon / init_range_lat)**2  
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 4-parameter spatial-only list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )
    ]

    # --- Define learning rates and parameter groups ---
    # 💥 REVISED: Set lr_fast from the global lr
    lr_fast = lr 
    fast_indices = [0, 1, 2, 3] # All parameters
    
    param_groups = [
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    # --- Print Job Info ---
    # Assuming lat_lon_resolution, v, mm_cond_number are defined globally
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Instantiate the Correct Class ---
    model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 REVISED: Call the Optimizer Method with Plateau settings ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-5, 
            scheduler_type='plateau', # Explicitly set to plateau
            patience=patience,        # Pass patience
            factor=factor             # Pass factor
        )

    # --- Call the Correct Fit Method ---
    out, epoch_ran = model_instance.fit_vecc_scheduler_fullbatch(
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_SPATIAL_log_reparam, 
            epochs=epochs
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results: {out}")

In [ ]:
def cal(a):
    day_indices = [0] 
    for day_idx in day_indices:  

        daily_hourly_map = daily_hourly_maps[day_idx]
        daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

        # Convert initial parameters to a list of 1-element tensors
        params_list = [
            torch.tensor([val], dtype=torch.float64, requires_grad=True, device=device_str) for val in a
        ]
        
        # 💡 CRITICAL: Concatenate the list into a single tensor
        # The full_likelihood function expects a single tensor, not a list
        params_tensor = torch.cat(params_list)

        # Assuming 'kernels_repar_space' has your 'fit_vecchia_adams' class
        model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
                smooth = v,
                input_map = daily_hourly_map,
                aggregated_data = daily_aggregated_tensor,
                nns_map = nns_map,
                mm_cond_number = mm_cond_number,
                nheads = nheads
            )
        
        # 💡 Pass the single 'params_tensor' and the correct 4-parameter spatial covariance function
        bb = model_instance.full_likelihood_avg(
            params = params_tensor, 
            input_data = daily_aggregated_tensor, 
            y = daily_aggregated_tensor[:,2], 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam
        )
     
        cov_map = model_instance.cov_structure_saver(params_tensor, model_instance.matern_cov_SPATIAL_log_reparam)
        vecchia_nll = model_instance.vecchia_space_fullbatch( # Change this to your chosen Vecchia implementation
            params = params_tensor, 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam, 
            cov_map = cov_map # Assuming cov_map is precomputed or computed internally
        )
 
    return bb, vecchia_nll



a = [3.5764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [3.6764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


a = [3.4764, 0.7940, 0.3087, 0.6157]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


Full Negative Log-Likelihood: (22936.561132578532, 22882.89402724899)
Full Negative Log-Likelihood: (22940.44941805631, 22893.58097275657)
Full Negative Log-Likelihood: (22959.63309778168, 22897.341313474775)
